In [43]:
# Step 0: setup 
sites = ['osceola']
site = sites[0]

# areas... each has 4 plots, each plot can have 5 clipplots
#numbers = ["A1", "A2", "B1", "B2", "C1", "C2", "D1", "E2"] 
areas = ["E2"] 

subplots = [1,2,3,4]  
#subplots = ["A"] # C1, C2

#lazpath = 'C:/Users/bdrye/Box/External 3D Fuels/data_NEW/' + site + '/tls/synoptic'
lazpath = 'D:/5x5/osc_laz/synoptic/' + areas[0] + '_synoptic' # local copy of files from Box 
laspath = 'D:/5x5/osc_las'

csvpath = 'C:/Users/bdrye/Box/External 3D Fuels/' + site + '/data/spatial/poles'

In [44]:
# Step 1: see which .laz and/or .csv files might be missing
# copy .laz files from C:/Users/bdrye/Box/External 3D Fuels/data_NEW/' + site + '/tls/synoptic'
# to D:/hb_laz 
# set sites, numbers for each location. get numbers from tracking spreadsheet
# note: change prefix lines: 21, 27
from os import listdir

for site in sites:
    print(site)
    lazFiles = listdir(lazpath)
    print(lazFiles)
    
    csvFiles = listdir(csvpath)
#    print(csvFiles)

    for n in areas:
        for sub in subplots:        
            csvFileName = n + '-' + str(sub) + '_5x5.csv'
            if not csvFileName in csvFiles:
                print('WARNING: csv not found: ' + csvFileName)

# if no warnings, the laz and Csv files exist for each numbered location

osceola
['Grid_76 - SINGLESCANS - 200114_164610.laz', 'Grid_77 - SINGLESCANS - 200114_185000.laz', 'Grid_78 - SINGLESCANS - 200114_184420.laz', 'Grid_79 - SINGLESCANS - 200114_181944.laz', 'Grid_80_E2_Center - SINGLESCANS - 200114_185604.laz', 'Grid_81 - SINGLESCANS - 200114_183852.laz', 'Grid_82 - SINGLESCANS - 200114_182441.laz', 'Grid_83 - SINGLESCANS - 200114_182916.laz', 'Grid_84 - SINGLESCANS - 200114_183409.laz']


In [45]:
# Step 2: convert laz files to las files
# 
# hb_laz -> hb_las

# create geoprocessing script
import arcgisscripting
gp = arcgisscripting.create()

# import lastools by using the geoprocessing script
gp.AddToolbox(r"C:\LAStools\ArcGIS_toolbox\LAStools.tbx")

# set wd 
arcpy.env.workspace = lazpath 

for file in arcpy.ListFiles("*.laz"):
    outfile = arcpy.Describe(file).baseName
    outfile = outfile.split()[0] # get rid of spaces 1_NE - SINGLESCANS - 220225_174518.laz
    print(outfile)
    arcpy.gp.laszip(file, 
                    False, 
                    False, 
                    "las", 
                    laspath + '/' + outfile,
                    laspath, 
                    "", 
                    "", 
                    True)


Grid_76
Grid_77
Grid_78
Grid_79
Grid_80_E2_Center
Grid_81
Grid_82
Grid_83
Grid_84


In [46]:
# Step 3: get pole locations, make polygon txt file, 
# clip las files and save as #_5x5.laz
import arcpy
import pandas as pd
import subprocess
from os import listdir

# this is what runs the LAStools command
def check_output(command,console):
    if console == True:
        process = subprocess.Popen(command)
    else:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    output,error = process.communicate()
    returncode = process.poll()
    return returncode,output 

for site in sites:
    print(site)
    lasFiles = listdir(laspath)
    #print(lasFiles)
    
    csvFiles = listdir(csvpath)

    for n in areas:
        print("n: " + str(n))
        cornerFiles = []
        for fn in lasFiles:
            if fn.endswith(".las"):
                cornerFiles.append(laspath + '/' + fn)
                
        for sub in subplots:
        
            # get polygon coordinates from csv file
            # create a txt file with format below... 
            csvFileName = n + '-' + str(sub) + '_5x5.csv'
            csvInfo = pd.read_csv(csvpath + '/' + csvFileName, sep=',')
            print(csvInfo)
            NE_X = csvInfo[(csvInfo.PlotName == 'NE') & (csvInfo.PlotType == 'Pole')].X
            NE_Y = csvInfo[(csvInfo.PlotName == 'NE') & (csvInfo.PlotType == 'Pole')].Y
            NW_X = csvInfo[(csvInfo.PlotName == 'NW') & (csvInfo.PlotType == 'Pole')].X
            NW_Y = csvInfo[(csvInfo.PlotName == 'NW') & (csvInfo.PlotType == 'Pole')].Y
            SW_X = csvInfo[(csvInfo.PlotName == 'SW') & (csvInfo.PlotType == 'Pole')].X
            SW_Y = csvInfo[(csvInfo.PlotName == 'SW') & (csvInfo.PlotType == 'Pole')].Y
            SE_X = csvInfo[(csvInfo.PlotName == 'SE') & (csvInfo.PlotType == 'Pole')].X
            SE_Y = csvInfo[(csvInfo.PlotName == 'SE') & (csvInfo.PlotType == 'Pole')].Y

            if len(NE_X) == 0 or len(NE_Y) == 0 or len(NW_X) == 0 or len(NW_Y) == 0 or len(SW_X) == 0 or len(SW_Y) == 0 or len(SE_X) == 0 or len(SE_Y) == 0:
                print('Error: CSV file is missing a pole. edit csv: change Clip to Pole')
                assert(False)

            with open(laspath + '/' + "corners.txt", "w") as file:
                file.write(str(NE_X.to_numpy()[0] + 3) + ' ' + str(NE_Y.to_numpy()[0] + 3) + "\n")
                file.write(str(NW_X.to_numpy()[0] - 3) + ' ' + str(NW_Y.to_numpy()[0] + 3) + "\n")
                file.write(str(SW_X.to_numpy()[0] - 3) + ' ' + str(SW_Y.to_numpy()[0] - 3) + "\n")
                file.write(str(SE_X.to_numpy()[0] + 3) + ' ' + str(SE_Y.to_numpy()[0] - 3) + "\n")
                file.write(str(NE_X.to_numpy()[0] + 3) + ' ' + str(NE_Y.to_numpy()[0] + 3) + "\n")

            # save a .shp file
            from pathlib import Path
            Path(laspath + "/shp").mkdir(parents=True, exist_ok=True)

            coords = [(NE_X.to_numpy()[0]+3, NE_Y.to_numpy()[0]+3), 
                      (NW_X.to_numpy()[0]-3, NW_Y.to_numpy()[0]+3),
                      (SW_X.to_numpy()[0]-3, SW_Y.to_numpy()[0]-3),
                      (SE_X.to_numpy()[0]+3, SE_Y.to_numpy()[0]-3),
                      (NE_X.to_numpy()[0]+3, NE_Y.to_numpy()[0]+3)
                      ]
            ar = arcpy.Array()
            for x, y in coords:
                ar.add(arcpy.Point(x, y))
            polyline = arcpy.Polyline(ar)
            arcpy.management.CopyFeatures(polyline, laspath + "/shp/" + str(n) + '-' + str(sub) + '_5x5.shp')

            # clip the .las files for this area-subplot using a shp file from the csv
            fileString = ""
            for cf in cornerFiles:
                fileString = fileString + ' ' + cf
            # print(fileString)
            cornerString = laspath + '/corners.txt' 
            #print(cornerString)
            outputString = laspath + '/' + n + '-' + str(sub) + '_5x5_3m.laz'   # something like 9_5x5.laz
            #print(outputString)
            callString = "C:/LAStools/bin/lasclip -i " + fileString + " -merged -poly " + cornerString + " -o " + outputString
            callString = callString.replace('/', '\\')
            print(callString)
            #subprocess.call(callString)
            check_output(callString, False)
            print(n + '-' + str(sub))

osceola
n: E2
   Unnamed: 0        id              X  ...  PlotName  PlotType      Dist
0           1  E2-1.laz  364380.738183  ...        NW      Pole  3.242490
1           2  E2-1.laz  364382.762465  ...       CTR      Clip  0.322089
2           3  E2-1.laz  364385.675130  ...        SE      Pole  3.516296
3           4  E2-1.laz  364381.484599  ...        SW      Clip  2.325214
4           5  E2-1.laz  364385.466054  ...        NE      Pole  3.517310
5           6  E2-1.laz  364380.637612  ...        SW      Pole  3.656507
6           7       NaN       0.000000  ...       NaN       NaN  0.000000
7           8       NaN       0.000000  ...       NaN       NaN  0.000000
8           9       NaN       0.000000  ...       NaN       NaN  0.000000

[9 rows x 10 columns]
C:\LAStools\bin\lasclip -i  D:\5x5\osc_las\Grid_76.las D:\5x5\osc_las\Grid_77.las D:\5x5\osc_las\Grid_78.las D:\5x5\osc_las\Grid_79.las D:\5x5\osc_las\Grid_80_E2_Center.las D:\5x5\osc_las\Grid_81.las D:\5x5\osc_las\Grid_82.

In [ ]:
# Step 4: copy files from D:\5x5\aucilla_las\*.laz to 

#outputPath = 'C:/Users/bdrye/Box/External 3D Fuels/data_NEW/' + site + '/tls/plots/5x5/biomass/scan'

In [213]:
# Step 5: normalize the *_5x5_3m.laz files, copy to biomass/norm

# Step 6: repeat (from step 0) for Scan Only .laz files

0    431126.939895
1    431129.503262
2    431130.517315
3    431125.839415
4    431126.870184
5    431128.176932
6    431130.831236
7    431129.292995
8    431125.525578
Name: X, dtype: float64
0    3.547890e+06
1    3.547890e+06
2    3.547894e+06
3    3.547888e+06
4    3.547892e+06
5    3.547891e+06
6    3.547889e+06
7    3.547892e+06
8    3.547893e+06
Name: Y, dtype: float64
